In [1]:
import pandas as pd
import numpy as np
import os.path
from pathlib import Path
import glob
import json

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

# Find the dataset @ (CSV files 2022-04-21, https://webrobots.io/kickstarter-datasets/)
# download link: https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2022-04-21T03_20_08_060Z.zip

### Create master Data Frame
before running this code, create a folder inside the repo folder called 'raw_data' contiaining the exctracted CSV files from the download link.

The code will create a new directory called 'data' to store the processed data.

In [2]:
# create folder inside the current directory to hold the processed data
try:
    Path('data').mkdir(parents=True, exist_ok=False)
    print ('creating data directory')
except:
    print ('directory already exists')
    pass

directory already exists


In [3]:
# path to each file in the all_data folder
all_paths = glob.glob('raw_data/*.csv')

# list to append df's to
list_of_df = []

# loop through all paths and append each csv as a df
for filename in all_paths:
    df = pd.read_csv(filename, index_col=None, header=0)
    list_of_df.append(df)

# concat all df's into one df
master_df = pd.concat(list_of_df, axis=0, ignore_index=True)

# save df as master csv
master_df.to_csv('data/master_df.csv', index=False)

### Unpack columns containing JSON objects
The code below takes all columns that represent dictionaries and saves them as individual csv files.

In [4]:
def make_json(string):
    '''
    converts the string representation of a json object into a python dict.
    returns np.nan if the string contains commas.
    '''
    try:
        # replace single quotations to make the string represent a JSON object
        json_acceptable_string = string.replace("'", "\"")
        return(json.loads(json_acceptable_string))
    except:
        # if the string is still not JSON compatible return np.nan
        return np.nan

In [5]:
def unpack(data_frame, column):
    '''
    unpacks dict in column to dataframe.
    '''
    # convert all strings into JSON objects
    unpacked = data_frame[column].apply(make_json).to_frame()

    # unpack JSON into DataFrame
    return pd.json_normalize(unpacked[column])

In [6]:
# define which columns are represented as dicts
unpack_list = ['category', 'creator', 'location', 'photo', 'profile', 'urls']

# init empty df 
unpacked_df = pd.DataFrame()

# unpack each column of dicts, save each as their own csv
for value in unpack_list:
    frame = unpack(master_df, value)
    frame.to_csv(f'data/{value}.csv', index=False)  

In [7]:
# read category data for name and parent name
df_category = pd.read_csv('data/category.csv')
df_category = df_category[['name', 'parent_name']]

In [8]:
# drop columns deemed unessecary
model_data = master_df.drop(columns = ['backers_count','usd_pledged','is_starrable','current_currency','static_usd_rate','usd_exchange_rate','usd_type','id','name', 'slug', 'category', 'creator', 'location', 'photo', 'profile', 'urls', 'country_displayable_name', 'currency_symbol', 'currency_trailing_code', 'disable_communication', 'source_url', 'currency', 'pledged', 'blurb'])

# create goal_usd column so that all goal amounts are in the same units, drop fx_rate and goals afterward
model_data['goal_usd'] = model_data['fx_rate']*model_data['goal']
model_data = model_data.drop(columns=['fx_rate', 'goal'])

# combine category data and all other date
model_data = pd.concat([model_data, df_category], axis=1, join='inner')

# create percentage funded column as another metric
model_data['percentage_funded'] = model_data.converted_pledged_amount/model_data.goal_usd*100

# create total days active column as another metric
model_data['total_days_active'] = (model_data.deadline-model_data.created_at)*0.00001157
model_data['launch_time'] = (model_data.launched_at-model_data.created_at)*0.00001157
model_data = model_data.drop(columns=['state_changed_at', 'deadline', 'launched_at'])
model_data

,converted_pledged_amount,country,created_at,spotlight,staff_pick,state,goal_usd,name,parent_name,percentage_funded,total_days_active,launch_time
0,5034,US,1609376406,True,False,successful,5000.000000,Cookbooks,Food,100.680000,29.994357,1.957378
1,48365,HK,1606278560,True,True,successful,44624.926500,Cookbooks,Food,108.381131,90.449007,30.470127
2,98,ES,1606477096,True,False,successful,54.095050,Cookbooks,Food,181.162603,16.466933,3.172101
3,127765,US,1604500905,True,True,successful,65000.000000,Cookbooks,Food,196.561538,64.574056,19.589896
4,14574,GB,1605454727,True,True,successful,13823.881138,Cookbooks,Food,105.426254,36.088774,6.099334
...,...,...,...,...,...,...,...,...,...,...,...,...
30987,3395,US,1434905504,True,False,successful,3000.000000,Fiction,Publishing,113.166667,104.846831,74.857391
30988,21112,US,1440643700,True,False,successful,12000.000000,Fiction,Publishing,175.933333,38.111337,8.121897
30989,2000,US,1441111356,True,False,successful,2000.000000,Fiction,Publishing,100.000000,60.256653,0.277773
30990,3125,US,1436479764,True,False,successful,3000.000000,Fiction,Publishing,104.166667,83.577804,53.588364


In [9]:
# init encoder for x values
encoder = LabelEncoder()

# encode boolean and string type columns
encoding_columns = (model_data.select_dtypes(include=['bool', 'object'])).columns
for column in encoding_columns:
    model_data[column] = encoder.fit_transform(model_data[column])

In [10]:
model_data

,converted_pledged_amount,country,created_at,spotlight,staff_pick,state,goal_usd,name,parent_name,percentage_funded,total_days_active,launch_time
0,5034,24,1609376406,1,0,3,5000.000000,7,4,100.680000,29.994357,1.957378
1,48365,11,1606278560,1,1,3,44624.926500,7,4,108.381131,90.449007,30.470127
2,98,7,1606477096,1,0,3,54.095050,7,4,181.162603,16.466933,3.172101
3,127765,24,1604500905,1,1,3,65000.000000,7,4,196.561538,64.574056,19.589896
4,14574,9,1605454727,1,1,3,13823.881138,7,4,105.426254,36.088774,6.099334
...,...,...,...,...,...,...,...,...,...,...,...,...
30987,3395,24,1434905504,1,0,3,3000.000000,13,7,113.166667,104.846831,74.857391
30988,21112,24,1440643700,1,0,3,12000.000000,13,7,175.933333,38.111337,8.121897
30989,2000,24,1441111356,1,0,3,2000.000000,13,7,100.000000,60.256653,0.277773
30990,3125,24,1436479764,1,0,3,3000.000000,13,7,104.166667,83.577804,53.588364


In [11]:
# Convert 'created at' to datetime format
import datetime
model_data['Date Created'] = pd.to_datetime(model_data['created_at'], unit='s')
model_data['Date Created'] = model_data['Date Created'].dt.strftime('%Y/%m/%d')
model_data

,converted_pledged_amount,country,created_at,spotlight,staff_pick,state,goal_usd,name,parent_name,percentage_funded,total_days_active,launch_time,Date Created
0,5034,24,1609376406,1,0,3,5000.000000,7,4,100.680000,29.994357,1.957378,2020/12/31
1,48365,11,1606278560,1,1,3,44624.926500,7,4,108.381131,90.449007,30.470127,2020/11/25
2,98,7,1606477096,1,0,3,54.095050,7,4,181.162603,16.466933,3.172101,2020/11/27
3,127765,24,1604500905,1,1,3,65000.000000,7,4,196.561538,64.574056,19.589896,2020/11/04
4,14574,9,1605454727,1,1,3,13823.881138,7,4,105.426254,36.088774,6.099334,2020/11/15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30987,3395,24,1434905504,1,0,3,3000.000000,13,7,113.166667,104.846831,74.857391,2015/06/21
30988,21112,24,1440643700,1,0,3,12000.000000,13,7,175.933333,38.111337,8.121897,2015/08/27
30989,2000,24,1441111356,1,0,3,2000.000000,13,7,100.000000,60.256653,0.277773,2015/09/01
30990,3125,24,1436479764,1,0,3,3000.000000,13,7,104.166667,83.577804,53.588364,2015/07/09


In [12]:
# Drop NA's
model_data = model_data.dropna()
model_data

,converted_pledged_amount,country,created_at,spotlight,staff_pick,state,goal_usd,name,parent_name,percentage_funded,total_days_active,launch_time,Date Created
0,5034,24,1609376406,1,0,3,5000.000000,7,4,100.680000,29.994357,1.957378,2020/12/31
1,48365,11,1606278560,1,1,3,44624.926500,7,4,108.381131,90.449007,30.470127,2020/11/25
2,98,7,1606477096,1,0,3,54.095050,7,4,181.162603,16.466933,3.172101,2020/11/27
3,127765,24,1604500905,1,1,3,65000.000000,7,4,196.561538,64.574056,19.589896,2020/11/04
4,14574,9,1605454727,1,1,3,13823.881138,7,4,105.426254,36.088774,6.099334,2020/11/15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30987,3395,24,1434905504,1,0,3,3000.000000,13,7,113.166667,104.846831,74.857391,2015/06/21
30988,21112,24,1440643700,1,0,3,12000.000000,13,7,175.933333,38.111337,8.121897,2015/08/27
30989,2000,24,1441111356,1,0,3,2000.000000,13,7,100.000000,60.256653,0.277773,2015/09/01
30990,3125,24,1436479764,1,0,3,3000.000000,13,7,104.166667,83.577804,53.588364,2015/07/09


In [13]:
# Create validation dataset
val_start_date = '2021/02/01'
val_end_date = '2021/08/07'
val_mask = (model_data['Date Created'] >= val_start_date) & (model_data['Date Created'] <= val_end_date)
val_dataset = model_data[val_mask]
val_dataset

,converted_pledged_amount,country,created_at,spotlight,staff_pick,state,goal_usd,name,parent_name,percentage_funded,total_days_active,launch_time,Date Created
96,2696,1,1625314596,1,0,3,407.706392,28,2,661.260174,30.477393,16.482321,2021/07/03
97,2949,14,1624865518,1,0,3,194.773500,28,2,1514.066339,81.117848,21.138968,2021/06/28
98,84407,3,1625252511,1,0,3,9930.636500,28,2,849.965659,51.606377,16.618697,2021/07/02
99,222818,24,1625077438,1,1,3,50000.000000,28,2,445.636000,48.520692,18.531252,2021/06/30
100,9715,3,1625034114,1,1,3,8897.850304,28,2,109.183675,61.629757,18.673193,2021/06/30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29726,1200,24,1618924995,0,0,1,5000.000000,27,5,24.000000,60.404992,0.426112,2021/04/20
29728,2,5,1618215205,0,0,1,5409.504950,19,10,0.036972,30.262295,0.272855,2021/04/12
29729,0,16,1617895322,0,0,1,19857.958785,40,5,0.000000,60.485623,1.036279,2021/04/08
29730,298,15,1616497370,0,0,1,5950.455445,41,5,5.008020,73.339048,13.235212,2021/03/23


In [14]:
# Split validation dataset into X and y variables
X = val_dataset.drop(columns=['state','Date Created'])
y = val_dataset.state

In [15]:
# generate 70% train/validation split 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3,
                                                     random_state = 1,
                                                     stratify = y)

In [16]:
# Instantiate a linear SVM model
from sklearn.svm import SVC
classifier = SVC(kernel='linear',max_iter=500000)
classifier

SVC(kernel='linear', max_iter=500000)

In [17]:
# Fit the data
classifier.fit(X_train, y_train)

C:\Users\Wai Fatt Yee\anaconda3\envs\py37\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


SVC(kernel='linear', max_iter=500000)

In [18]:
# Score the accuracy
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_val, y_val)}")

Training Data Score: 0.8387297633872977
Testing Data Score: 0.8388969521044993


In [19]:
# Make predictions using the test data
predictions = classifier.predict(X_val)
results = pd.DataFrame({
    "Prediction": predictions, 
    "Actual": y_val
}).reset_index(drop=True)
results.tail()

,Prediction,Actual
684,3,3
685,3,3
686,3,1
687,3,3
688,3,3


In [20]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, predictions)
cm_df = pd.DataFrame(
cm, index=['Actual 0', 'Actual 1', 'Actual 2', 'Actual 3'], columns=['Predicted 0', 'Predicted 1', 'Predicted 2', 'Predicted 3']
)
# Displaying results
print("Confusion Matrix")
display(cm_df)

Confusion Matrix


,Predicted 0,Predicted 1,Predicted 2,Predicted 3
Actual 0,0,0,0,11
Actual 1,0,1,0,91
Actual 2,0,0,0,9
Actual 3,0,0,0,577


In [21]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_val, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       1.00      0.01      0.02        92
           2       0.00      0.00      0.00         9
           3       0.84      1.00      0.91       577

    accuracy                           0.84       689
   macro avg       0.46      0.25      0.23       689
weighted avg       0.84      0.84      0.77       689



C:\Users\Wai Fatt Yee\anaconda3\envs\py37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Wai Fatt Yee\anaconda3\envs\py37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Wai Fatt Yee\anaconda3\envs\py37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
# Create train/test dataset
train_test_dataset = model_data.loc[val_mask == False]
train_test_dataset = train_test_dataset.drop(['Date Created'], axis=1)
train_test_dataset

,converted_pledged_amount,country,created_at,spotlight,staff_pick,state,goal_usd,name,parent_name,percentage_funded,total_days_active,launch_time
0,5034,24,1609376406,1,0,3,5000.000000,7,4,100.680000,29.994357,1.957378
1,48365,11,1606278560,1,1,3,44624.926500,7,4,108.381131,90.449007,30.470127
2,98,7,1606477096,1,0,3,54.095050,7,4,181.162603,16.466933,3.172101
3,127765,24,1604500905,1,1,3,65000.000000,7,4,196.561538,64.574056,19.589896
4,14574,9,1605454727,1,1,3,13823.881138,7,4,105.426254,36.088774,6.099334
...,...,...,...,...,...,...,...,...,...,...,...,...
30987,3395,24,1434905504,1,0,3,3000.000000,13,7,113.166667,104.846831,74.857391
30988,21112,24,1440643700,1,0,3,12000.000000,13,7,175.933333,38.111337,8.121897
30989,2000,24,1441111356,1,0,3,2000.000000,13,7,100.000000,60.256653,0.277773
30990,3125,24,1436479764,1,0,3,3000.000000,13,7,104.166667,83.577804,53.588364


In [23]:
# split into X and y variables
X = train_test_dataset.drop(columns=['state'])
y = train_test_dataset.state#.to_frame()

In [24]:
# generate 70% train/test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                       random_state = 1,
                                                       stratify = y)

In [25]:
# init standard scaler
scaler = StandardScaler()

# scale Train and testing set, columns=X.columns to keep column headers.
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
X_train_scaled.head()

,converted_pledged_amount,country,created_at,spotlight,staff_pick,goal_usd,name,parent_name,percentage_funded,total_days_active,launch_time
0,-0.046637,0.622086,-0.612076,-1.277071,-0.450349,-0.002407,-0.903791,-0.455475,-0.014432,-0.355266,-0.334502
1,-0.046540,-1.325784,-0.741280,-1.277071,-0.450349,0.051478,1.026783,1.017933,-0.014431,1.054969,1.081655
2,-0.046556,0.622086,-0.037092,-1.277071,-0.450349,-0.035710,0.103465,-0.087123,-0.014094,0.678030,0.796673
3,-0.046430,-1.845216,1.049544,0.783042,-0.450349,-0.035882,-0.735915,0.649581,-0.008436,-0.274410,-0.315809
4,-0.043847,0.622086,-0.973126,-1.277071,-0.450349,-0.031891,-1.491356,-0.823827,-0.013851,2.491364,2.523556


In [26]:
# Instantiate a linear SVM model
from sklearn.svm import SVC
classifier = SVC(kernel='linear',max_iter=500000)
classifier

SVC(kernel='linear', max_iter=500000)

In [27]:
# Fit the data
classifier.fit(X_train, y_train)

C:\Users\Wai Fatt Yee\anaconda3\envs\py37\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=500000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


SVC(kernel='linear', max_iter=500000)

In [28]:
# Score the accuracy
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.5635983471897247
Testing Data Score: 0.5632984901277585


In [29]:
# Make predictions using the test data
predictions = classifier.predict(X_test)
results = pd.DataFrame({
    "Prediction": predictions, 
    "Actual": y_test
}).reset_index(drop=True)
results.tail()

,Prediction,Actual
8605,3,3
8606,3,3
8607,1,2
8608,3,3
8609,3,3


In [30]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
cm, index=['Actual 0', 'Actual 1', 'Actual 2', 'Actual 3'], columns=['Predicted 0', 'Predicted 1', 'Predicted 2', 'Predicted 3']
)
# Displaying results
print("Confusion Matrix")
display(cm_df)

Confusion Matrix


,Predicted 0,Predicted 1,Predicted 2,Predicted 3
Actual 0,7,8,0,261
Actual 1,25,139,0,2605
Actual 2,0,218,2,8
Actual 3,89,544,2,4702


In [31]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.06      0.03      0.04       276
           1       0.15      0.05      0.08      2769
           2       0.50      0.01      0.02       228
           3       0.62      0.88      0.73      5337

    accuracy                           0.56      8610
   macro avg       0.33      0.24      0.21      8610
weighted avg       0.45      0.56      0.48      8610

